## Contents
* [Handling of Missing Data](#handling-of-missing-data)
  * [Missing Data in Pandas](#missing-data-in-pandas)
  * [Null Value Operations](#null-value-operations)
* [Transforming Data](#transforming-data)
  * [Sorting DataFrame Value](#sorting-dataframe-values)
  * [Add Columns / Rows](#add-columns--rows)
  * [Remove Columns / Rows](#remove-columns--rows)
* [Aggregating DataFrames](#aggregating-dataframes)
  * [Descriptive Statistics](#descriptive-statistics)
* [Grouped Summary statistics](#grouped-summary-statistics)
  * [Pivot Table](#pivot-table)
* [Joining DataFrames](#joining-dataframes)

Pandas is a data analysis and manipulation library which provides three fundamental data structures: `Index`, `Series`, and `DataFrame`. For more details on the data structures and an overview of the Pandas library see the previous article - [Introduction to the Pandas Library](/quick%20start/pandas-introduction.html){: .post__link}. 

The focus of this article is to service as a introduction and guide to the basic functions for modifying and manipulating the core data structures, including handling of missing data, `DataFrame` transformation, aggregating data, and joining `DataFrames`

---

<br>

## Handling of Missing Data
Datasets are typically not clean and ready for use, including that they may contain some level of missing data. How missing data is indicated can vary depending on the data source which can add an additional level of complication when handling the missing data.

Generally, there is no single optimal choice when developing a scheme to indicate the presence of missing data in a data table. Typically the schemes center around using a mask that indicates missing values or implementing a sentinel value that indicates a missing value.

### Missing Data in Pandas
Within Pandas missing data is indicated by one of the two Python null values: `NaN` (Not a Number) indicating a missing floating-point value or the `None` object, and pandas can convert between the two when needed.

In [2]:
import pandas as pd
import numpy as np

series_1 = pd.Series([10, np.nan, 15, None])
series_1

0    10.0
1     NaN
2    15.0
3     NaN
dtype: float64

>For types that do not have a sentinel value (e.g. integer) Pandas automatically type-casts when a missing value is present (e.g. cast an integer to a floating point)

In [3]:
series_2 = pd.Series([0, 1, 2])
series_2

0    0
1    1
2    2
dtype: int64

In [4]:
series_2[1] = None
series_2

0    0.0
1    NaN
2    2.0
dtype: float64

>Missing values will propagate through arithmetic operations between pandas objects. Descriptive statistics and certain computational methods ([Series](https://pandas.pydata.org/pandas-docs/dev/reference/series.html#api-series-stats){: .post__link} and [DataFrame](https://pandas.pydata.org/pandas-docs/dev/reference/frame.html#api-dataframe-stats){: .post__link}) all account for missing data (e.g. when summing data missing values will be treated as zero).

### Null Value Operations
Pandas offer various methods for identifying, removing, and replacing missing values within the data structures.
#### Identifying Null or Missing Values
`pandas.isna(obj)`
* A functions that takes a scalar or array-like object and is used to indicate whether there are missing values.
* Returns a scalar or array of boolean values indicating whether each element is missing

In [5]:
pd.isna('string')

False

In [6]:
pd.isna(np.nan)

True

In [7]:
series = pd.Series([2, 4, np.nan, 8])
pd.isna(series)

0    False
1    False
2     True
3    False
dtype: bool

`DataFrame.isna()`

* A function used to identify missing values within a Series or DataFrame 
* Returns a boolean same-sized object indicating whether an element is a missing value 
  * `None` or `NaN` get mapped to `True` and everything else is mapped to `False`
* `.isnull()` is an alias for `.isna()`
* `.notna()` performs the opposite operation of `.isna()`
  * Non-missing values get mapped to `True`
* Boolean masks, like the object returned by `isna()`, can be used directly as a `Series` or `DataFrame` index
  * For more detail see: [Data Subsetting - Pandas](/quick%20start/pandas-data-selection.html){: .post__link}

>Characters such as an empty string `' '` or `np.inf` are not considered missing values unless `pandas.options.mod.use_inf_as_na = True` is set

In [8]:
# Series Examples
even_series = pd.Series([2, None, 6, np.nan, 10])
even_series.isna()

0    False
1     True
2    False
3     True
4    False
dtype: bool

In [9]:
df = pd.DataFrame({'name':['Oliver', 'John', 'Jane', 'Ashley', 'Steve'],
                    'A':[71, 63, 86, 98, 75],
                    'B':[83, np.nan, 91, 99, 77],
                    'C':[np.nan, 87, 92, 79, 82],
                    'D':[88, 77, 78, 60, np.nan]})
# Example DataFrame
df

,name,A,B,C,D
0,Oliver,71,83.0,NaN,88.0
1,John,63,NaN,87.0,77.0
2,Jane,86,91.0,92.0,78.0
3,Ashley,98,99.0,79.0,60.0
4,Steve,75,77.0,82.0,NaN


In [10]:
df.isna()

,name,A,B,C,D
0,False,False,False,True,False
1,False,False,True,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,True


In [11]:
# DataFrame - Determine if an element is NOT missing a value
# (opposite of .isna())
df.notna()

,name,A,B,C,D
0,True,True,True,False,True
1,True,True,False,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,False


#### Removing Null or Missing Values
`Series.dropna(axis=0, inplace=False, how=None)`

* A function that returns a new series with missing values removed
  * axis (0 or 'index'): a Series has only one axis to drop values from
  * inplace (bool): if True the operation of removing missing values is done inplace and .dropna() returns None
  * how (str): not currently in use and is kept for compatibility

In [12]:
# Example Series
series = df['B']
series

0    83.0
1     NaN
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [13]:
# Default arguments
series.dropna()

0    83.0
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [14]:
# Reprinting the series verifies that the series has not 
# been modified
series

0    83.0
1     NaN
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [15]:
# Set inplace=True to complete teh operation inplace
series.dropna(inplace=True)
series

0    83.0
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

`DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)`
* axis {0/'index' or 1/'column'}: determine if missing values are removed from rows or columns
  * 0: drop rows that contain missing values
  * 1: drop columns that contain missing values
* how {'any', 'all'}: determines if a row or column is removed when there is at least one missing value
or all missing values
* thresh (int): specify the number of non-missing values required to drop a row or column
* subset (column or sequence labels): labels along an other axis (e.g. when dropping rows a list of columns to consider)
* inplace (bool): boolean value, if `True` the operation of removing missing values is done inplace and returns None

>Dropping missing values from a DataFrame operates on an entire row or column, a single missing value cannot be dropped

In [16]:
# Example Series
series = df['B']
series

0    83.0
1     NaN
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [17]:
# Default arguments
series.dropna()

0    83.0
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [18]:
# Reprinting the series verifies that the series has not 
# been modified
series

0    83.0
1     NaN
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [19]:
# Set inplace=True to complete teh operation inplace
series.dropna(inplace=True)
series

0    83.0
2    91.0
3    99.0
4    77.0
Name: B, dtype: float64

In [20]:
# DataFrame Example
df

,name,A,B,C,D
0,Oliver,71,83.0,NaN,88.0
1,John,63,NaN,87.0,77.0
2,Jane,86,91.0,92.0,78.0
3,Ashley,98,99.0,79.0,60.0
4,Steve,75,77.0,82.0,NaN


In [21]:
# Default: Drop all rows with any missing values
df.dropna()

,name,A,B,C,D
2,Jane,86,91.0,92.0,78.0
3,Ashley,98,99.0,79.0,60.0


In [22]:
# Set axis=1 to drop all columns with missing values
df.dropna(axis=1)

,name,A
0,Oliver,71
1,John,63
2,Jane,86
3,Ashley,98
4,Steve,75


In [23]:
# Set how keyword to drop only row/columns where all values are missing
df['F']=np.nan
df

,name,A,B,C,D,F
0,Oliver,71,83.0,NaN,88.0,NaN
1,John,63,NaN,87.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,NaN,NaN


In [24]:
df.dropna(axis='columns', how='all')

,name,A,B,C,D
0,Oliver,71,83.0,NaN,88.0
1,John,63,NaN,87.0,77.0
2,Jane,86,91.0,92.0,78.0
3,Ashley,98,99.0,79.0,60.0
4,Steve,75,77.0,82.0,NaN


In [25]:
# Set thresh keyword to specify the minimum number of non-missing values
df['F'] = [79, np.nan, 88, np.nan, np.nan]
df

,name,A,B,C,D,F
0,Oliver,71,83.0,NaN,88.0,79.0
1,John,63,NaN,87.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,NaN,NaN


In [26]:
df.dropna(axis='columns', thresh=3)

,name,A,B,C,D
0,Oliver,71,83.0,NaN,88.0
1,John,63,NaN,87.0,77.0
2,Jane,86,91.0,92.0,78.0
3,Ashley,98,99.0,79.0,60.0
4,Steve,75,77.0,82.0,NaN


#### Filling Null or Missing Values
`DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)`
* value (scalar, dict, `Series`, `DataFrame`): the value used to replace missing values. Passing in a dict/`Series`/`DataFrame`
of values specifying which value to use for each index (for a `Series`), or column (for a `DataFrame`), values not in the dict/`Series`/`DataFrame` will
not be replaced
* method {'backfill', 'bfill', 'pad', 'ffill', None}: the method used for replacing missing values in reindexed series
  * pad/ffill: propagate the last valid observation forward to the next
  * backfill/bfill: uses the next valid observation to replace missing values
* axis: the axis to replace missing values
  * 0 or 'index' for a `Series`
  * 0/'index' or 1/'column' for a `DataFrame`
* inplace (bool): boolean value, if `True` the operation of removing missing values is done inplace and returns None
* limit (int): 
  * If method is specified, limit sets the maximum number of consecutive missing values to forward or backwards fill
  * If method is not specified, limit sets the maximum number of missing values along the entire axis that will be filled
* downcast (dict): a dictionary of item->dtype of what downcast or string 'infer' which will try to downcast to an appropriate equal type

In [27]:
# Example DataFrame
df

,name,A,B,C,D,F
0,Oliver,71,83.0,NaN,88.0,79.0
1,John,63,NaN,87.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,NaN,NaN


In [28]:
# Basic fill methods
df.fillna(value=0)

,name,A,B,C,D,F
0,Oliver,71,83.0,0.0,88.0,79.0
1,John,63,0.0,87.0,77.0,0.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,0.0
4,Steve,75,77.0,82.0,0.0,0.0


In [29]:
# Fill NaN using the backfill method - If the last value is NaN it will remain NaN
df.fillna(method='bfill')

,name,A,B,C,D,F
0,Oliver,71,83.0,87.0,88.0,79.0
1,John,63,91.0,87.0,77.0,88.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,NaN,NaN


In [30]:
# Fill Nan using the forwardfill method - If the first value is NaN it will remain NaN
df.fillna(method='ffill')

,name,A,B,C,D,F
0,Oliver,71,83.0,NaN,88.0,79.0
1,John,63,83.0,87.0,77.0,79.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,88.0
4,Steve,75,77.0,82.0,60.0,88.0


In [31]:
# Forward fill and set axis to rows (compare with previous example)
df.fillna(method='ffill', axis=1)

,name,A,B,C,D,F
0,Oliver,71,83.0,83.0,88.0,79.0
1,John,63,63,87.0,77.0,77.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,60.0
4,Steve,75,77.0,82.0,82.0,82.0


In [32]:
# Set value to a dictionary of values (unique replacement value for each column)
values = {'A':222, 'B':333, 'C':444, 'D':555, 'F':666}
df.fillna(value=values)

,name,A,B,C,D,F
0,Oliver,71,83.0,444.0,88.0,79.0
1,John,63,333.0,87.0,77.0,666.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,666.0
4,Steve,75,77.0,82.0,555.0,666.0


In [33]:
# Set limit to 1 with no method to fill only the first missing value
df.fillna(value = values, limit=1)

,name,A,B,C,D,F
0,Oliver,71,83.0,444.0,88.0,79.0
1,John,63,333.0,87.0,77.0,666.0
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,555.0,NaN


---
## Transforming Data

### Sorting DataFrame Values
The two methods covered in this section, `.sort_values()` and `.sort_index()`, allow for efficient sorting of the Pandas `DataFrame` data structure.

`DataFrame.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index='False', key=None)`
* by (string or list of strings): Name or list of names to sort by
  * 0 or 'index': `by` may contain index levels and/or column labels
  * 1 or 'columns': `by` may contain column levels and/or index labels
* axis(0/'index' or 1/'columns'): axis to be sorted
* ascending (bool or list of bools), default `True`: If `True` sort values in ascending order, if `False` sort in descending order
* inplace (bool): if `True` perform the operation in-place
* kind ('quicksort', 'mergesort', 'heapsort', 'stable'): sorting algorithm used (see [numpy.sort()](https://numpy.org/doc/stable/reference/generated/numpy.sort.html#numpy.sort){: .post__link} for details)
* na_position ('first' or 'last'): argument specifying where `NaNs` should appear
* ignore_index (bool): If `True` the result axis will be labeled 0, 1, ..., n-1
* key (callable): when passed in the key function is applied to the series values before sorting

In [34]:
# Example DataFrame
df

,name,A,B,C,D,F
0,Oliver,71,83.0,NaN,88.0,79.0
1,John,63,NaN,87.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN
4,Steve,75,77.0,82.0,NaN,NaN


In [35]:
# Default Sorting Options - Sort by Column A
df.sort_values(by='A')

,name,A,B,C,D,F
1,John,63,NaN,87.0,77.0,NaN
0,Oliver,71,83.0,NaN,88.0,79.0
4,Steve,75,77.0,82.0,NaN,NaN
2,Jane,86,91.0,92.0,78.0,88.0
3,Ashley,98,99.0,79.0,60.0,NaN


In [36]:
# Sort in descending order and ignore index
df.sort_values(by='A', ascending=False, ignore_index=True)

,name,A,B,C,D,F
0,Ashley,98,99.0,79.0,60.0,NaN
1,Jane,86,91.0,92.0,78.0,88.0
2,Steve,75,77.0,82.0,NaN,NaN
3,Oliver,71,83.0,NaN,88.0,79.0
4,John,63,NaN,87.0,77.0,NaN


In [37]:
# Sort by multiple columns - Sorting occurs in order listed
# Sort ascending can be set for each item in list passed to by
df.sort_values(by=['name', 'A'], ascending=[True, False], ignore_index=True)

,name,A,B,C,D,F
0,Ashley,98,99.0,79.0,60.0,NaN
1,Jane,86,91.0,92.0,78.0,88.0
2,John,63,NaN,87.0,77.0,NaN
3,Oliver,71,83.0,NaN,88.0,79.0
4,Steve,75,77.0,82.0,NaN,NaN


In [38]:
# Add new column of string values - for key argument example
df['G'] = ['a', 'B', 'C', 'd', 'E']
df

,name,A,B,C,D,F,G
0,Oliver,71,83.0,NaN,88.0,79.0,a
1,John,63,NaN,87.0,77.0,NaN,B
2,Jane,86,91.0,92.0,78.0,88.0,C
3,Ashley,98,99.0,79.0,60.0,NaN,d
4,Steve,75,77.0,82.0,NaN,NaN,E


In [39]:
df.sort_values(by='G')

,name,A,B,C,D,F,G
1,John,63,NaN,87.0,77.0,NaN,B
2,Jane,86,91.0,92.0,78.0,88.0,C
4,Steve,75,77.0,82.0,NaN,NaN,E
0,Oliver,71,83.0,NaN,88.0,79.0,a
3,Ashley,98,99.0,79.0,60.0,NaN,d


In [40]:
# Sorting a DataFrame with a key function - function evaluated then column sorted
df.sort_values(by='G', key=lambda col: col.str.lower())

,name,A,B,C,D,F,G
0,Oliver,71,83.0,NaN,88.0,79.0,a
1,John,63,NaN,87.0,77.0,NaN,B
2,Jane,86,91.0,92.0,78.0,88.0,C
3,Ashley,98,99.0,79.0,60.0,NaN,d
4,Steve,75,77.0,82.0,NaN,NaN,E


In [41]:
# Default sorting with NaNs
df.sort_values(by='F')

,name,A,B,C,D,F,G
0,Oliver,71,83.0,NaN,88.0,79.0,a
2,Jane,86,91.0,92.0,78.0,88.0,C
1,John,63,NaN,87.0,77.0,NaN,B
3,Ashley,98,99.0,79.0,60.0,NaN,d
4,Steve,75,77.0,82.0,NaN,NaN,E


In [42]:
# Set the na_position argument
df.sort_values(by='F', na_position='first')

,name,A,B,C,D,F,G
1,John,63,NaN,87.0,77.0,NaN,B
3,Ashley,98,99.0,79.0,60.0,NaN,d
4,Steve,75,77.0,82.0,NaN,NaN,E
0,Oliver,71,83.0,NaN,88.0,79.0,a
2,Jane,86,91.0,92.0,78.0,88.0,C


`DataFrame.sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True, ignore_index='False', key=None)`
* axis(0/'index' or 1/'columns'): axis to be sorted
* level (int or level name or lists of ints or names): sort on values in specified index level(s) when not `None`
* ascending (bool or list of bools): If `True` sort values in ascending order, if `False` sort in descending order
* inplace (bool): if `True` perform the operation in-place
* kind ('quicksort', 'mergesort', 'heapsort', 'stable'): sorting algorithm used (see [numpy.sort()](https://numpy.org/doc/stable/reference/generated/numpy.sort.html#numpy.sort){: .post__link} for details)
* na_position ('first' or 'last'): argument specifying where `NaNs` should appear
* sort_remaining (boo): when `True` and sorting by level and index is multilevel, sort by other level as well, in order, after sorting by specified level
* ignore_index (bool): if `True` the result axis will be labeled 0, 1, ..., n-1
* key (callable): when passed in the key function is applied to the series values before sorting

In [43]:
# Set the name field to the DataFrame index
df.set_index('name', inplace=True)
df

,A,B,C,D,F,G
name,,,,,,
Oliver,71,83.0,NaN,88.0,79.0,a
John,63,NaN,87.0,77.0,NaN,B
Jane,86,91.0,92.0,78.0,88.0,C
Ashley,98,99.0,79.0,60.0,NaN,d
Steve,75,77.0,82.0,NaN,NaN,E


In [44]:
# Default sort_index
df.sort_index()

,A,B,C,D,F,G
name,,,,,,
Ashley,98,99.0,79.0,60.0,NaN,d
Jane,86,91.0,92.0,78.0,88.0,C
John,63,NaN,87.0,77.0,NaN,B
Oliver,71,83.0,NaN,88.0,79.0,a
Steve,75,77.0,82.0,NaN,NaN,E


In [45]:
# Sort index in descending order
df.sort_index(ascending=False)

,A,B,C,D,F,G
name,,,,,,
Steve,75,77.0,82.0,NaN,NaN,E
Oliver,71,83.0,NaN,88.0,79.0,a
John,63,NaN,87.0,77.0,NaN,B
Jane,86,91.0,92.0,78.0,88.0,C
Ashley,98,99.0,79.0,60.0,NaN,d


In [46]:
# Create and sort by hierarchial index
df['group'] = [1, 1, 2, 2, 1]
df.reset_index(inplace=True)
df.set_index(['group', 'name'], inplace=True)
df

A     B     C     D     F  G
group name                                 
1     Oliver  71  83.0   NaN  88.0  79.0  a
      John    63   NaN  87.0  77.0   NaN  B
2     Jane    86  91.0  92.0  78.0  88.0  C
      Ashley  98  99.0  79.0  60.0   NaN  d
1     Steve   75  77.0  82.0   NaN   NaN  E

In [47]:
# Sort outer index with default sort_remaining=True
df.sort_index(level='group')

A     B     C     D     F  G
group name                                 
1     John    63   NaN  87.0  77.0   NaN  B
      Oliver  71  83.0   NaN  88.0  79.0  a
      Steve   75  77.0  82.0   NaN   NaN  E
2     Ashley  98  99.0  79.0  60.0   NaN  d
      Jane    86  91.0  92.0  78.0  88.0  C

In [48]:
# Set argument sort_remaining=False - compare the order of names in each group 
# with previous example
df.sort_index(level='group', sort_remaining=False)

A     B     C     D     F  G
group name                                 
1     Oliver  71  83.0   NaN  88.0  79.0  a
      John    63   NaN  87.0  77.0   NaN  B
      Steve   75  77.0  82.0   NaN   NaN  E
2     Jane    86  91.0  92.0  78.0  88.0  C
      Ashley  98  99.0  79.0  60.0   NaN  d

In [49]:
# Sort by inner group
df.sort_index(level='name')

A     B     C     D     F  G
group name                                 
2     Ashley  98  99.0  79.0  60.0   NaN  d
      Jane    86  91.0  92.0  78.0  88.0  C
1     John    63   NaN  87.0  77.0   NaN  B
      Oliver  71  83.0   NaN  88.0  79.0  a
      Steve   75  77.0  82.0   NaN   NaN  E

In [50]:
# Change the sort axis to column labels
df.sort_index(axis=1, ascending=False)

G     F     D     C     B   A
group name                                 
1     Oliver  a  79.0  88.0   NaN  83.0  71
      John    B   NaN  77.0  87.0   NaN  63
2     Jane    C  88.0  78.0  92.0  91.0  86
      Ashley  d   NaN  60.0  79.0  99.0  98
1     Steve   E   NaN   NaN  82.0  77.0  75

### Add Columns / Rows

#### Add Columns to a `DataFrame`
Pandas offers a number of methods for adding columns of data to a `DataFrame`. The values of the new column can be given 
as an array or list of the same size as the `DataFrame` and then assigned to the new column by providing the name.

This is a common approach when the new column can be added to the end of the `DataFrame` (i.e. the last column)

In [51]:
df['E'] = [89, 77, 96, 83, 72]
df

A     B     C     D     F  G   E
group name                                     
1     Oliver  71  83.0   NaN  88.0  79.0  a  89
      John    63   NaN  87.0  77.0   NaN  B  77
2     Jane    86  91.0  92.0  78.0  88.0  C  96
      Ashley  98  99.0  79.0  60.0   NaN  d  83
1     Steve   75  77.0  82.0   NaN   NaN  E  72

`DataFrame.insert(loc, column, value, allow_duplicates=False)`<br>
A method the provides the flexibility to add a column in any position as well as providing options for inserting the column values
* loc (int): the insertion index, must be greater than or equal to 0 and less than or equal to the length of the DataFrame
* column (string, number, or hashable object): label of the inserted column
* value (scalar, series, or array-like): the values to be inserted
* allow_duplicates (bool): boolean value indicating whether to allow duplicates or not

In [52]:
# Drop E added in previous example
df.drop(columns=['E'], inplace=True)

# Insert E in proper location
e_values = [89, 77, np.nan, 83, 72]
df.insert(4, 'E', e_values)
df

A     B     C     D     E     F  G
group name                                       
1     Oliver  71  83.0   NaN  88.0  89.0  79.0  a
      John    63   NaN  87.0  77.0  77.0   NaN  B
2     Jane    86  91.0  92.0  78.0   NaN  88.0  C
      Ashley  98  99.0  79.0  60.0  83.0   NaN  d
1     Steve   75  77.0  82.0   NaN  72.0   NaN  E

`DataFrame.assign(**kwargs)`<br>
A method that assigns new columns to a `DataFrame` and returns a new object with the original columns in addition to the newly defined columns.
* `**kwargs` (dict of {str: callable or Series}): The column names are the key words of the dictionary
  * If values are callable, pandas will compute on the `DataFrame` and assign to the new columns

>Multiple columns can be assigned using `.assign()` and later items in `**kwargs` can reference previously created new columns (i.e. items are computed/assigned in order)

In [53]:
df.assign(mean=df.mean(axis=1))

/tmp/ipykernel_282/3084921118.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.assign(mean=df.mean(axis=1))


A     B     C     D     E     F  G  mean
group name                                             
1     Oliver  71  83.0   NaN  88.0  89.0  79.0  a  82.0
      John    63   NaN  87.0  77.0  77.0   NaN  B  76.0
2     Jane    86  91.0  92.0  78.0   NaN  88.0  C  87.0
      Ashley  98  99.0  79.0  60.0  83.0   NaN  d  83.8
1     Steve   75  77.0  82.0   NaN  72.0   NaN  E  76.5

In [54]:
# Use assign to compute a new column that depends on a previously created new column
df.assign(H=[65, 55, 56, 62, 68], h=lambda x: x['H'] + 10)

A     B     C     D     E     F  G   H   h
group name                                               
1     Oliver  71  83.0   NaN  88.0  89.0  79.0  a  65  75
      John    63   NaN  87.0  77.0  77.0   NaN  B  55  65
2     Jane    86  91.0  92.0  78.0   NaN  88.0  C  56  66
      Ashley  98  99.0  79.0  60.0  83.0   NaN  d  62  72
1     Steve   75  77.0  82.0   NaN  72.0   NaN  E  68  78

#### Add Rows to a `DataFrame`
`DataFrame.loc[]`<br>
Similar to above, `.loc[]` can also be utilized to add rows to a `DataFrame`

In [57]:
# DataFrame Cleanup
df.reset_index(inplace=True)
df.drop(columns=['G', 'group'], inplace=True)
# DataFrame Example
df

,name,A,B,C,D,E,F,new_col
0,Oliver,71,83.0,NaN,88.0,89.0,79.0,new value
1,John,63,NaN,87.0,77.0,77.0,NaN,new value
2,Jane,86,91.0,92.0,78.0,NaN,88.0,new value
3,Ashley,98,99.0,79.0,60.0,83.0,NaN,new value
4,Steve,75,77.0,82.0,NaN,72.0,NaN,new value


In [58]:
df.loc[5] = ['Aaron', 88, np.nan, 93, 72, 97, 85]
df

ValueError: cannot set a row with mismatched columns

`pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=True)`

A function that provides the ability to concatenate pandas objects along a particular axis with optional set logic along the other axes
* objs (sequence or mapping of `Series` or `DataFrame` objects): If mapping is passed, the sorted keys will be used as the `keys` argument unless it is passed in.
* axis (0/'index' or 1/'columns'): the axis to concatenate along
* join ('inner', 'outer'): how to handle indexes on other axes
* ignore_index (bool): If `True` do not use the index value along the concatenation axis, typically helpful when the concatenation axis does not have meaningful indexing information
* keys (sequence): when multiples levels are passed in they should contain tuples, hierarchical indexes are constructed using the passed keys as the outermost level
* levels (list of sequences): unique values to use for constructing a multi-index
* names (list): names for the levels in the hierarchical index
* verify_integrity (bool): If `True` pandas will check whether the new concatenated axis contains duplicates
* sort (bool): sort non-concatenation axis if it is not aligned when join is 'outer'.
* copy (bool): If `False` data is not copied unnecessarily

>For more joining operations see [Joining DataFrames](#joining-dataframes){: .post__link}

In [ ]:
# Example DataFrames
df

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0


In [ ]:
df2 = pd.DataFrame({'name':['Katie', 'Bob'],
                    'A':[92, 88],
                    'B':[91, 81],
                    'C':[99, np.nan],
                    'D':[87, 74],
                    'E':[79, 78],
                    'F':[79, 78]})
df2

,name,A,B,C,D,E,F
0,Katie,92,91,99.0,87,79,79
1,Bob,88,81,NaN,74,78,78


In [ ]:
# Concat with default arguments, append df2 to df
pd.concat([df, df2])

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0
0,Katie,92,91.0,99.0,87.0,79.0,79.0
1,Bob,88,81.0,NaN,74.0,78.0,78.0


In [ ]:
# Ignore Index when it has no meaningful information, labels will be 0, 1, .., n-1
pd.concat([df, df2], ignore_index=True)

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0
6,Katie,92,91.0,99.0,87.0,79.0,79.0
7,Bob,88,81.0,NaN,74.0,78.0,78.0


In [ ]:
# Construct hierarchical index
pd.concat([df, df2], keys=['One', 'Two'])

name   A     B     C     D     E     F
One 0  Oliver  71  83.0   NaN  88.0  89.0  79.0
    1    John  63   NaN  87.0  77.0  77.0   NaN
    2    Jane  86  91.0  92.0  78.0   NaN  88.0
    3  Ashley  98  99.0  79.0  60.0  83.0   NaN
    4   Steve  75  77.0  82.0   NaN  72.0   NaN
    5   Aaron  88   NaN  93.0  72.0  97.0  85.0
Two 0   Katie  92  91.0  99.0  87.0  79.0  79.0
    1     Bob  88  81.0   NaN  74.0  78.0  78.0

In [ ]:
# Default 'outer' join - includes all columns in either DataFrame
# Columns outside the intersection will be fill with NaN (Column G below)
df2['G'] = [84, 69]
df2
pd.concat([df, df2])

,name,A,B,C,D,E,F,G
0,Oliver,71,83.0,NaN,88.0,89.0,79.0,NaN
1,John,63,NaN,87.0,77.0,77.0,NaN,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0,NaN
3,Ashley,98,99.0,79.0,60.0,83.0,NaN,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0,NaN
0,Katie,92,91.0,99.0,87.0,79.0,79.0,84.0
1,Bob,88,81.0,NaN,74.0,78.0,78.0,69.0


In [ ]:
# Set join to 'inner', to return only columns shared by both objects
pd.concat([df, df2], join='inner')

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0
0,Katie,92,91.0,99.0,87.0,79.0,79.0
1,Bob,88,81.0,NaN,74.0,78.0,78.0


`.concat()` can also be used to add columns to a `DataFrame` by changing the axis parameter

In [ ]:
G=[98, 88, 82, 76, 88]
df3 = pd.DataFrame({'G':G})
pd.concat([df, df3], axis=1)

,name,A,B,C,D,E,F,G
0,Oliver,71,83.0,NaN,88.0,89.0,79.0,98.0
1,John,63,NaN,87.0,77.0,77.0,NaN,88.0
2,Jane,86,91.0,92.0,78.0,NaN,88.0,82.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN,76.0
4,Steve,75,77.0,82.0,NaN,72.0,NaN,88.0
5,Aaron,88,NaN,93.0,72.0,97.0,85.0,NaN


### Remove Columns / Rows
`DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')`
`.drop()` is a method which removes rows or columns by providing label names and the corresponding axis or by specifying the index or column names.
* labels (single label or list-like): the index or column label to drop
* axis (0/'index' or 1/'column'): drop labels from teh index or columns
* index (single label or list-like): can be used as an alternative to specifying the axis parameter
  * `.drop(labels, axis=0)` is equivalent to `.drop(index=labels)`
* columns (single label or list-list): can be used as an alternative to specifying the axis parameter
  * `.drop(labels, axis=1)` is equivalent to `.drop(columns=labels)`
* level (int or level name): the level from which the labels will be removed
* inplace (bool): if `True` the drop operation occurs in place `.drop()` returns `None` otherwise a copy of the `DataFrame` is returned
* errors ('ignore', 'raise'): if 'ignore' errors will be suppressed and only existing labels are dropped from the `DataFrame`

In [ ]:
# Example DataFrame
df

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0


In [ ]:
# Drop Columns
df.drop(labels=['B', 'E', 'F'], axis=1)

,name,A,C,D
0,Oliver,71,NaN,88.0
1,John,63,87.0,77.0
2,Jane,86,92.0,78.0
3,Ashley,98,79.0,60.0
4,Steve,75,82.0,NaN
5,Aaron,88,93.0,72.0


In [ ]:
df.drop(columns=['B', 'E'])

,name,A,C,D,F
0,Oliver,71,NaN,88.0,79.0
1,John,63,87.0,77.0,NaN
2,Jane,86,92.0,78.0,88.0
3,Ashley,98,79.0,60.0,NaN
4,Steve,75,82.0,NaN,NaN
5,Aaron,88,93.0,72.0,85.0


In [ ]:
# Drop Rows by Index
df.drop([0, 3])

,name,A,B,C,D,E,F
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0


In [ ]:
# Drop columns/rows for MultiIndex DataFrame
df['grouping'] = [1, 1, 2, 1, 2, 2]
df.set_index(['grouping', 'name'], inplace=True)
df

A     B     C     D     E     F
grouping name                                    
1        Oliver  71  83.0   NaN  88.0  89.0  79.0
         John    63   NaN  87.0  77.0  77.0   NaN
2        Jane    86  91.0  92.0  78.0   NaN  88.0
1        Ashley  98  99.0  79.0  60.0  83.0   NaN
2        Steve   75  77.0  82.0   NaN  72.0   NaN
         Aaron   88   NaN  93.0  72.0  97.0  85.0

In [ ]:
# Drop a index combination
df.drop(index=(1, 'John'))

A     B     C     D     E     F
grouping name                                    
1        Oliver  71  83.0   NaN  88.0  89.0  79.0
2        Jane    86  91.0  92.0  78.0   NaN  88.0
1        Ashley  98  99.0  79.0  60.0  83.0   NaN
2        Steve   75  77.0  82.0   NaN  72.0   NaN
         Aaron   88   NaN  93.0  72.0  97.0  85.0

---

## Aggregating DataFrames
An essential step to gaining insights into a dataset is the ability to effectively summarize the data. Pandas offers a variety of methods ranging from simple calculations (e.g. `.sum()`) to more complex operations using `groupby`
### Descriptive Statistics
Descriptive statistics are operations that summarize the central tendency, dispersion, and shape of a dataset

Basic summary methods include `.count()`, `.sum()`, `.mode()`, `.min()`, `max()`, `.var()`, `.std()`, `.mean()`, `.quantile()`, `.median()`

`DataFrame.sum()`<br>
`DataFrame.mean()`<br>
`DataFrame.median()`<br>
`DataFrame.min()`<br>
`DataFrame.max()`<br>
A few general parameters explored here include
* axis(0, 1): specify the axis (index or columns) the function should be applied on
* skipna (bool): if `True` exclude null values when computing
* level (int or level name): if the axis is hierarchical, count along a specified level,  collapsing into a scalar
* numeric_only (bool): includes only float, int, and boolean columns, when `None` (default) pandas attempts to use everything then only numeric data
* `**kwargs`: any additional keyword arguments
* min_count (int): specify the required number of valid value needed to perform the operation
  * parameter for `.sum()`

In [ ]:
# Example DataFrame
df.reset_index(inplace=True)
df.drop(columns=['grouping'], inplace=True)
df

,name,A,B,C,D,E,F
0,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,John,63,NaN,87.0,77.0,77.0,NaN
2,Jane,86,91.0,92.0,78.0,NaN,88.0
3,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,Steve,75,77.0,82.0,NaN,72.0,NaN
5,Aaron,88,NaN,93.0,72.0,97.0,85.0


In [ ]:
df.mean()

/tmp/ipykernel_11941/3698961737.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


A    80.166667
B    87.500000
C    86.600000
D    75.000000
E    83.600000
F    84.000000
dtype: float64

In [ ]:
# Set the axis parameter to 1 to operate on the rows of the DataFrame
df.set_index('name').mean(axis=1)

name
Oliver    82.0
John      76.0
Jane      87.0
Ashley    83.8
Steve     76.5
Aaron     87.0
dtype: float64

In [ ]:
df.min()

name    Aaron
A          63
B        77.0
C        79.0
D        60.0
E        72.0
F        79.0
dtype: object

In [ ]:
# Set the numeric_only parameter to exclude the name column from the operation
df.min(numeric_only=True)

A    63.0
B    77.0
C    79.0
D    60.0
E    72.0
F    79.0
dtype: float64

In [ ]:
df.sum(numeric_only=True)

A    481.0
B    350.0
C    433.0
D    375.0
E    418.0
F    252.0
dtype: float64

In [ ]:
# Set the min_count parameter to only perform the operation
# when there is at least the specified number of values
df.sum(min_count=5, numeric_only=True)

A    481.0
B      NaN
C    433.0
D    375.0
E    418.0
F      NaN
dtype: float64

#### Method: .describe()
A helpful method for generating descriptive statistics is `.describe()`<br>
`DataFrame.describe(percentiles=None, include=None, exclude=None, datatime_is_numeric=False)`
* percentiles (list-like): the percentiles to include in the output, and should be between 0 and 1, default = [.25, .5, .75]
* include ('all', list-like, or None): list of data types to include in the result
* exclude (list-like of dtypes or None): data types to omit from the result
* datetime_is_numeric (bool): boolean value indicating whether datetime data types should be treated as numeric

In [ ]:
df.describe()

,A,B,C,D,E,F
count,6.000000,4.000000,5.000000,5.000000,5.000000,3.000000
mean,80.166667,87.500000,86.600000,75.000000,83.600000,84.000000
std,12.797135,9.574271,6.107373,10.198039,9.838699,4.582576
min,63.000000,77.000000,79.000000,60.000000,72.000000,79.000000
25%,72.000000,81.500000,82.000000,72.000000,77.000000,82.000000
50%,80.500000,87.000000,87.000000,77.000000,83.000000,85.000000
75%,87.500000,93.000000,92.000000,78.000000,89.000000,86.500000
max,98.000000,99.000000,93.000000,88.000000,97.000000,88.000000


#### Method: .agg()
The `.agg()` (alias of `.aggregate()`) method aggregates values using one or more operations over a specified axis.<br>
`DataFrame.agg(func=None, axis=0, *args, **kwargs)`
* func (function, str, list, or dict): the function(s) to be used for aggregating the data
* axis (0/'index', 1/'columns'): the axis the operation is performed on
  * 'index': the function is applied to each column
  * 'columns': the function is applied to each row
* `*args`: positional arguments passed to `func`
* `**kwargs`: keyword arguments passed to `func`

In [ ]:
# Remove the name column and calculate column sum, mean, and max
df.drop('name',axis=1).agg(['sum', 'mean', 'max'])

,A,B,C,D,E,F
sum,481.000000,350.0,433.0,375.0,418.0,252.0
mean,80.166667,87.5,86.6,75.0,83.6,84.0
max,98.000000,99.0,93.0,88.0,97.0,88.0


Pandas offers many other computation and descriptive statistic methods. For more information and example see the [Pandas Documentation](https://pandas.pydata.org/docs/reference/frame.html#computations-descriptive-stats)

---

## Grouped Summary Statistics
The basic descriptive statistics can provide quick insights into the dataset, however to to conditionally aggregate by a label or index requires the use of the `groupby` operation.

The `groupby` operation can be though of as consisting of three stages:
* Split: the breaking up and grouping of the `DataFrame`
* Apply: computing the desired operation(s) for each group
* Combine: merge the results of the of the operation(s)

When using the `.groupby()` method these steps are not explicit providing relief from considering *how* the computation is done and focusing on the operation as a whole 

`DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_key=True, squeeze=NoDefault.no_default, observed=False, dropna=True)`

Parameters explored here include:
* by (mapping, function, label, or list of labels): determines the grouping
* level (int, level name, sequence of level names): specify a particular level(s) to group by if the axis hierarchical  
* dropna (bool): when `True` missing values will be dropped, when `False` missing values will be treated as the key in groups

In [ ]:
# Example DataFrame
df['class'] = ['MWF', 'MWF', 'TTH', 'MWF', 'TTH', 'MWF']
df

,name,A,B,C,D,E,F,class
0,Oliver,71,83.0,NaN,88.0,89.0,79.0,MWF
1,John,63,NaN,87.0,77.0,77.0,NaN,MWF
2,Jane,86,91.0,92.0,78.0,NaN,88.0,TTH
3,Ashley,98,99.0,79.0,60.0,83.0,NaN,MWF
4,Steve,75,77.0,82.0,NaN,72.0,NaN,TTH
5,Aaron,88,NaN,93.0,72.0,97.0,85.0,MWF


In [ ]:
# Group by Column Value
df.groupby(['class']).mean()

,A,B,C,D,E,F
class,,,,,,
MWF,80.0,91.0,86.333333,74.25,86.5,82.0
TTH,80.5,84.0,87.000000,78.00,72.0,88.0


In [ ]:
# Group by hierarchical index
# Add second level column and set index
df['time'] = ['M', 'N', 'N', 'M', 'M', 'N']
df.set_index(['class', 'time'], inplace=True)
df

name   A     B     C     D     E     F
class time                                          
MWF   M     Oliver  71  83.0   NaN  88.0  89.0  79.0
      N       John  63   NaN  87.0  77.0  77.0   NaN
TTH   N       Jane  86  91.0  92.0  78.0   NaN  88.0
MWF   M     Ashley  98  99.0  79.0  60.0  83.0   NaN
TTH   M      Steve  75  77.0  82.0   NaN  72.0   NaN
MWF   N      Aaron  88   NaN  93.0  72.0  97.0  85.0

In [ ]:
df.groupby(level='time').mean()

,A,B,C,D,E,F
time,,,,,,
M,81.333333,86.333333,80.500000,74.000000,81.333333,79.0
N,79.000000,91.000000,90.666667,75.666667,87.000000,86.5


In [ ]:
df.groupby(level=0).mean()

,A,B,C,D,E,F
class,,,,,,
MWF,80.0,91.0,86.333333,74.25,86.5,82.0
TTH,80.5,84.0,87.000000,78.00,72.0,88.0


In [ ]:
# Handling missing values in the by column
df.reset_index(inplace=True)
df['class']=['MWF', None, None, 'MWF', 'TTH', 'MWF']
df

,class,time,name,A,B,C,D,E,F
0,MWF,M,Oliver,71,83.0,NaN,88.0,89.0,79.0
1,None,N,John,63,NaN,87.0,77.0,77.0,NaN
2,None,N,Jane,86,91.0,92.0,78.0,NaN,88.0
3,MWF,M,Ashley,98,99.0,79.0,60.0,83.0,NaN
4,TTH,M,Steve,75,77.0,82.0,NaN,72.0,NaN
5,MWF,N,Aaron,88,NaN,93.0,72.0,97.0,85.0


In [ ]:
df.groupby(by='class').mean()

,A,B,C,D,E,F
class,,,,,,
MWF,85.666667,91.0,86.0,73.333333,89.666667,82.0
TTH,75.000000,77.0,82.0,NaN,72.000000,NaN


In [ ]:
df.groupby(by='class', dropna=False).mean()

,A,B,C,D,E,F
class,,,,,,
MWF,85.666667,91.0,86.0,73.333333,89.666667,82.0
TTH,75.000000,77.0,82.0,NaN,72.000000,NaN
NaN,74.500000,91.0,89.5,77.500000,77.000000,88.0


### Pivot Table
Similar to `.groupby` the `.pivot_table()` method provide a a multidimensional summary of the dataset. While `.groupby` splits/combines data across a 1D index, `pivot_table` splits/combines the data across a 2D grid. <br>

`pandas.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)`

Parameters explored here include:
* values: the columns to aggregate - the column to summarize
* index (column, Grouper, array, or a list of them): what to group by columns. If an array, it must be the same length as the `DataFrame`
* columns (column, Grouper, array, or list of them): If an array, it must be the same length as the `DataFrame`
* aggfunc (function, list of functions, dict): if a list of functions the pivot table will contain a hierarchical column whose top level will be the function names, if a dict is passed the key is the column to summarize and value is the function or list of functions
* fill_value (scalar): specify a value to replace missing values in the resulting pivot table

In [ ]:
# Example DataFrame
df['group'] = ['group1', 'group2', 'group1', 'group2', 'group2', 'group2']
df

,class,time,name,A,B,C,D,E,F,group
0,MWF,M,Oliver,71,83.0,NaN,88.0,89.0,79.0,group1
1,None,N,John,63,NaN,87.0,77.0,77.0,NaN,group2
2,None,N,Jane,86,91.0,92.0,78.0,NaN,88.0,group1
3,MWF,M,Ashley,98,99.0,79.0,60.0,83.0,NaN,group2
4,TTH,M,Steve,75,77.0,82.0,NaN,72.0,NaN,group2
5,MWF,N,Aaron,88,NaN,93.0,72.0,97.0,85.0,group2


In [ ]:
df.pivot_table(values='A', 
                index=['class'])

,A
class,
MWF,85.666667
TTH,75.000000


In [ ]:
# Pass a list to the index parameter
df.pivot_table(values='A', 
                index=['class', 'time'])

A
class time      
MWF   M     84.5
      N     88.0
TTH   M     75.0

In [ ]:
# Include the summarization of multiple columns
df.pivot_table(values=['A', 'B'], 
                index=['class', 'time'])

A     B
class time            
MWF   M     84.5  91.0
      N     88.0   NaN
TTH   M     75.0  77.0

In [ ]:
# Pass a list of functions to aggfunc
df.pivot_table(values=['A', 'B'], 
                index=['class', 'time'], 
                aggfunc=[np.min, np.max])

amin       amax      
              A     B    A     B
class time                      
MWF   M      71  83.0   98  99.0
      N      88   NaN   88   NaN
TTH   M      75  77.0   75  77.0

In [ ]:
# Pivot on two variables - provide a column name to column
df.pivot_table(values=['A', 'B'], 
                index=['class', 'time'], 
                aggfunc=[np.min, np.max], 
                columns='group')

amin                        amax                     
                A             B             A             B       
group      group1 group2 group1 group2 group1 group2 group1 group2
class time                                                        
MWF   M      71.0   98.0   83.0   99.0   71.0   98.0   83.0   99.0
      N       NaN   88.0    NaN    NaN    NaN   88.0    NaN    NaN
TTH   M       NaN   75.0    NaN   77.0    NaN   75.0    NaN   77.0

In [ ]:
# Specify a fill value for NaN in pivot table
df.pivot_table(values=['A', 'B'], 
                index=['class', 'time'], 
                aggfunc=[np.min, np.max], 
                columns='group', 
                fill_value=0.0)

amin                        amax                     
                A             B             A             B       
group      group1 group2 group1 group2 group1 group2 group1 group2
class time                                                        
MWF   M        71     98     83     99     71     98     83     99
      N         0     88      0      0      0     88      0      0
TTH   M         0     75      0     77      0     75      0     77

---

## Joining DataFrames

A key feature offered by Pandas is the ability to conduct in-memory join and merge operations. This feature if implemented through the `pd.merge()` function and the related `.join()` method of `Series` and `DataFrame` objects.

The `pd.merge()` function implements 3 types of joins
* one-to-one: every row in the left table is related to one and only one row in the right table
* many-to-one: every row in the left table is related to one or more rows in the right table
* many-to-many: multiple rows in the left table is related to multiple rows in the right table

All three are performed with the same call to the function `pd.merge()` and the type of join performed is depended on the form of the input data

>If both key columns contain rows where the key is a null value these rows will be matched - this is different behavior than the usual SQL join behavior

`pandas.merge()` is a function that can be used to perform database-style joins on `Series` and `DataFrame` objects.

`pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)`
* left: `DataFrame`
* right: `DataFrame` or named `Series`
* how ('left', 'right', 'outer', 'inner', 'cross'): the type of join to perform
* on (label or list): column or index level names to join on, must be found in both `DataFrames`.
* left_on (label or list, or array-like): column or index level names to join on in the left `DataFrame`
* right_on (label or list, or array-like): column or index level names to join on in the right `DataFrame`
* left_index (bool): use the index from the left `DataFrame` as the join key(s)
* right_index (bool): use the index from the right `DataFrame as the join key(s)
* sort (bool): sort the join key in the resulting `DataFrame`
* suffixes (list-like): a length-2 sequence with each element indicating the suffix to add to overlapping column names in the left and right `DataFrames`. `None` can be passed in for either the right or left to leave the corresponding column name as-is.
* copy (bool): if `False` avoid copy when possible
* indicator (bool or str): if `True` add a column to the resulting `DataFrame` called `_merged` containing information on the source of each row.
* validate (str): if specified checks if the merge is of the specified type ("one_to_one", "one_to_many", or "many_to_one")

`DataFrame.merge()` is a method that can be called on a `DataFrame` object and provides the same join functionality with similar arguments. Using this method only the right `DataFrame` has to be specified.

In [ ]:
# Example DataFrames
df1 = pd.DataFrame({
    'id':['AAA', 'BBB'],
    'value': ['A1', 'B1']
})
df2 = pd.DataFrame({
    'id':['AAA', 'BBB'],
    'value': ['A2', 'B2']
})
print(f'df1:\n{df1}')
print(f'df2:\n{df2}')

df1:
    id value
0  AAA    A1
1  BBB    B1
df2:
    id value
0  AAA    A2
1  BBB    B2


In [ ]:
# Basic Merge - on column with same name
pd.merge(df1, df2, on='id')

,id,value_x,value_y
0,AAA,A1,A2
1,BBB,B1,B2


In [ ]:
# Basic Merge - using DataFrame.merge()
df1.merge(df2, on='id')

,id,value_x,value_y
0,AAA,A1,A2
1,BBB,B1,B2


In [ ]:
df1.rename(columns={'id':'left_id'}, inplace=True)
df2.rename(columns={'id':'right_id'}, inplace=True)

In [ ]:
# Merge with different column names
df1.merge(df2, 
            left_on='left_id', 
            right_on='right_id')

,left_id,value_x,right_id,value_y
0,AAA,A1,AAA,A2
1,BBB,B1,BBB,B2


In [ ]:
# Specify suffixes appended to overlapping columns (i.e value)
df1.merge(df2, 
            left_on='left_id', 
            right_on='right_id',
            suffixes=('_left', '_right'))

,left_id,value_left,right_id,value_right
0,AAA,A1,AAA,A2
1,BBB,B1,BBB,B2


In [ ]:
# Specify suffixes appended to overlapping columns (i.e value)
# Leave left DataFrame column as-is
df1.merge(df2, 
            left_on='left_id', 
            right_on='right_id',
            suffixes=(None, '_right'))

,left_id,value,right_id,value_right
0,AAA,A1,AAA,A2
1,BBB,B1,BBB,B2


In [ ]:
# Example DataFrame 3
df3 = pd.DataFrame({
    'right_id':['BBB', 'CCC'],
    'value': ['B3', 'C3']
})
print(f'd3:\n{df3}')

d3:
  right_id value
0      BBB    B3
1      CCC    C3


In [ ]:
# Join Type - Inner
df1.merge(df3, 
            left_on='left_id', 
            right_on='right_id', 
            how='inner')

,left_id,value_x,right_id,value_y
0,BBB,B1,BBB,B3


In [ ]:
# Join Type - Left
df1.merge(df3, 
            left_on='left_id', 
            right_on='right_id', 
            how='left')

,left_id,value_x,right_id,value_y
0,AAA,A1,NaN,NaN
1,BBB,B1,BBB,B3


In [ ]:
# Join Type - Outer
df1.merge(df3, 
            left_on='left_id', 
            right_on='right_id', 
            how='outer')

,left_id,value_x,right_id,value_y
0,AAA,A1,NaN,NaN
1,BBB,B1,BBB,B3
2,NaN,NaN,CCC,C3


The related `DataFrame.join()` method provides similar functionality as `.merge()` when joining a `DataFrame` to another `DataFrame` either on the index or a key column.

`DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)`

For additional information and examples please see the Pandas Documentation here: [pandas.DataFrame.join()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html?highlight=join#pandas.DataFrame.join)

If you enjoy what you read and find it helpful please check back, and check back often, click here [Medium](https://medium.com/@emguyant) and follow me while giving a clap to the article! Also don't forget to subscribe to the Inquisitive Nature publication.